# What to eat?

A chatbot you can use when you are not sure what to eat.

### 1. Import Libraries & Text Preprocessing

This cell loads required libraries such as `re`, `json`, `nltk`, and configures text preprocessing (stopwords, stemming).

In [1]:
import json
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# Note: This script requires ChromeDriver installed and added to your system PATH
from websitecrawl import crawl_google_maps_from_url # Extracts top 5 Google Maps results using headless Chrome with Selenium
from urllib.parse import quote
# import threading
# import time

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /Users/jm_jeon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jm_jeon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Preprocess user input: tokenize, remove stopwords, and stem
def preprocess(text):
    """
    Lowercase the input, remove punctuation, tokenize,
    remove stopwords, and apply stemming.
    """

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = nltk.word_tokenize(text)  # Tokenize the sentence
    filtered_tokens = [stemmer.stem(w) for w in tokens if w not in stop_words]  # Remove stopwords and stem
    return ' '.join(filtered_tokens)  # Return the processed string

### 2. Intent Handling

Load `intents.json` file and define a function to match user input to the closest intent using regex patterns.

In [3]:
# Load predefined intents from a JSON file
def load_intents(file_path='intents.json'):
    with open(file_path, 'r') as file:
        return json.load(file)['intents']

# Match user input to one of the intent patterns using regex
def match_intent(intents, user_input):
    for intent in intents:
        for pattern in intent["patterns"]:
            if re.search(pattern, user_input, re.IGNORECASE):  # Case-insensitive pattern matching
                return intent  # Return the matched intent
    return None  # No intent matched

### 4. Learning from Unknown Input

This function allows the chatbot to ask the user for the correct tag when an unknown phrase is encountered, and updates the `intents.json` file accordingly.

In [4]:
# Function to learn a new pattern from the user and store it in the intents JSON file
def learn_new_pattern(user_input, intents_file='intents.json'):
    # Ask user if they want to teach the bot
    print("Bot: I didn't understand that. Would you like to teach me this sentence?")
    consent = input("You (yes/no): ").strip().lower()
    if consent != "yes":
        return

    # Ask user which category (intent tag) this pattern should belong to
    print("Please choose a category:")
    print("1. Greeting")
    print("2. Food Recommendation")
    print("3. Location Search")
    print("4. Goodbye")
    choice = input("Enter the number or name of the category: ").strip().lower()

    # Map user choice to intent tag
    category_map = {
        "1": "greeting",
        "2": "recommend_food",
        "3": "search_food_places",
        "4": "goodbye",
        "greeting": "greeting",
        "food recommendation": "recommend_food",
        "recommendation": "recommend_food",
        "location search": "search_food_places",
        "goodbye": "goodbye"
    }

    tag = category_map.get(choice)
    if not tag:
        print("Invalid selection. Learning cancelled.")
        return

    # Try loading existing intents from file
    try:
        with open(intents_file, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        data = {"intents": []}

    # Preprocess user input using NLTK to normalize and reduce variation
    processed_input = preprocess(user_input)

    found = False
    for intent in data["intents"]:
        if intent["tag"] == tag:
            # Add new pattern only if it doesn't already exist
            if processed_input not in intent["patterns"]:
                intent["patterns"].append(processed_input)
            found = True
            break

    # If intent tag not found, create a new one
    if not found:
        new_intent = {
            "tag": tag,
            "patterns": [processed_input],
            "responses": ["Got it!"],
            "keywords": []
        }
        data["intents"].append(new_intent)

    # Save updated intents back to file
    with open(intents_file, 'w') as file:
        json.dump(data, file, indent=4)

    # Confirm learning to the user
    print(f"Bot: Learned the new pattern (as '{processed_input}') under tag '{tag}'. Thank you!")

### 5. Chatbot Main Loop

Main function to run the chatbot. Handles input, intent detection, response generation, crawling, and learning.

In [1]:
# Main chatbot loop
def chatbot():
    print("Welcome to 'What to Eat'!")

    # Store the last recommended food and last intent tag
    conversation_state = {
        "last_food": None,
        "last_tag": None
    }

    while True:
        user_input = input("You: ").strip()
        # This print code is to test in Microsoft Visual Studio Code
        # print("You:", user_input)

        # Load intents and match user input
        intents = load_intents()
        intent = match_intent(intents, user_input)

        if intent:
            tag = intent['tag']
            conversation_state['last_tag'] = tag

            # If intent is to recommend food
            if tag == 'recommend_food':
                index = random.randint(0, len(intent['responses']) - 1)
                response = intent['responses'][index]
                keyword = intent['keywords'][index]
                conversation_state['last_food'] = keyword.lower()  # Save keyword for future reference
                print("Bot:", response)

            # If intent is to search for food places
            elif tag == 'search_food_places':
                keyword = conversation_state.get('last_food')
                if keyword:
                    print("Bot:", random.choice(intent['responses']))
                    search_url = f"https://www.google.com/maps/search/{quote(keyword)}+singapore"
                    places = crawl_google_maps_from_url(search_url)  # Call external crawler
                    if places:
                        print("Here are some places you can check out:")
                        for place in places:
                            print("-", place)
                    else:
                        print("Sorry, I couldn't find any locations.")
                else:
                    print("Bot: I haven't recommended any food yet. Ask me for a recommendation first!")

            # If user says goodbye, exit the loop
            elif tag == 'goodbye':
                response = random.choice(intent['responses'])
                print("Bot:", response)
                break

            # Handle other known intents
            else:
                response = random.choice(intent['responses'])
                print("Bot:", response)

        # If no intent matched, try to learn from input
        else:
            learn_new_pattern(user_input)
            continue

In [ ]:
chatbot()

Welcome to 'What to Eat'!


### 6. Automated Chatbot Test (Simulated User Input)
This test simulates a conversation with the chatbot, including greeting, asking for food recommendations, requesting locations, and ending the chat. It also tests the chatbot’s response to unknown input.

In [9]:
from unittest.mock import patch

test_inputs = [
    "hi",
    "what to eat",
    "where can I eat it",
    "blargh blargh",
    "no",
    "bye"
]

with patch('builtins.input', side_effect=test_inputs):
    chatbot()

Welcome to 'What to Eat'!
Bot: Welcome!
Bot: Shawarma could be a delicious choice.
Bot: Sure, give me a moment to check nearby spots.
Here are some places you can check out:
- Cappadocia Turkish & Mediterranean Restaurant
- Zorba The Greek Taverna
- Marmaris Restaurant
- Shawarma Kingdom
- Cappadocia Turkish & Mediterranean Restaurant
Bot: I didn't understand that. Would you like to teach me this sentence?
Bot: Goodbye
